### Задача (варіант 1)

**Ієрархія:** p = 3, повна, m = (4, 3, 3)

<img src="images/lab_2_hierarchy.png" width=256>

**Метод розрахунку глобальних ваг:** дистрибутивний, ГВБВПА

*Методи аналізу альтернатив рішень на основі ієрархічної моделі критеріїв (МАІ, analytic hierarchy process, AHP)* складаються з наступних чотирьох загальних етапів:

1. Побудова ієрархічної моделі критеріїв, цілей та інших факторів, які впливають на головну ціль прийняття рішення; побудова множини альтернативних варіантів рішень.

2. Отримання суджень експертів щодо парних порівнянь елементів одного рівня ієрархії відносно спільного елементу вищого рівня. Парні порівняння проводяться у вибраній шкалі і за результатами будуються матриці парних порівнянь (МПП), які є обернено симетричними.

3. Математична обробка суджень експертів:
   * розрахунок локальних ваг елементів кожного рівня ієрархії відповідно до батьківських елементів вищого рівня на основі МПП; побудова локальних ранжувань;
   * аналіз узгодженості експертних оцінок;
   * розрахунок глобальних ваг елементів ієрархії відносно головної цілі прийняття рішення, використовуючи методи агрегування; побудова ранжування на основі глобальних ваг.

4. Аналіз чутливості отриманих ранжувань.

Постановка задачі:

**Дано:** 
 * $A = \{A_i | i = 1, \dots, N \}$ - множина альтернативних варіантів рішень;
 * $C = \{C_j | j = 1, \dots, M \}$ - множина критеріїв оцінювання альтернатив;
 * $a_{ij}$ - ненормована вага льтернативи $A_i$ за критерієм $C_j$;
 * $w_j^C$ - вага критерію $C_j, \sum_{j=1}^{M} w_j^C = 1$

**Потрібно:**
 * знайти глобальні ваги $w_i^{глоб}$ альтернатив $A_i, i = 1, \dots, N$

In [1]:
import numpy as np

In [2]:
# number of alternatives
n = 4
# number of criterions C
m = 2

Функція для генерування коректної матриці парних порівнянь для мультиплікативної групи

In [3]:
def _generate_random_list(n, low=1, high=9):
    """
    Generates a list consisting from 'n' elementswith uniform sampled values between 'low' and 
    'high' including and their multiplicative inverse elements.
    """
    elements = list(range(low, high + 1))
    elements_all = elements[:]
    for el in elements[1:]:
        elements_all.append(1 / el)
    prob_list = [1 / len(elements_all)] * len(elements_all)
    res = np.random.choice(elements_all, n, p=prob_list)
    return res

def generate_matched_matrix(n, low=1, high=9):
    """
    Generates matched multiplicational matrix.
    """
    mmp = np.zeros((n, n))
    mmp[0, :] = _generate_random_list(n, low=low, high=high)
    j = 0
    for i in range(1, n):
        mmp[i, j] = 1 / mmp[j, i]
    for i in range(1, n):
        for j in range(1, n):
            mmp[i, j] = mmp[i, 0] * mmp[0, j]
    mmp[0][0] = 1
    return mmp

# Data generation

Генеруємо матриці парних порівнянь, та ваги критеріїв.

In [4]:
# put some fixed seed for reproducibility of the results
np.random.seed(42)
# matrices of pair comparisons according to each of 'm' criterion
D = np.empty((m, n, n))
for i in range(m):
    d[i, :, :] = generate_matched_matrix(n)
    
# weights of criterions
w_crit = np.random.randn(m, 1)
w_crit = w_crit / np.sum(w_crit)

NameError: name 'd' is not defined

Обраховуємо локальні ваги використовуючи підхід EM з попередньої лабораторної роботи

In [5]:
# # generate unnormed local weights of alternatives with respect to all criterions
# # rows are corresponding to alternatives, columns - to criterions
# v = np.random.randn(n, m)
from numpy import linalg
def calculate_local(D):
    """
    D: rank 3 ndarray of shape (m, n, n). m - number of criterions, n - number of alternatives
    Returns (m, n) ndarray where each row represents local weights according to i-th criterion (i in [0, m-1])
    """
    m, n = D.shape[0], D.shape[1]
    res = np.empty((m, n))
    for idx in range(m):
        w, v = linalg.eig(D[idx, :, :])
        v_max = np.real(v[np.argmax(w)])
        res[idx, :] = v_max
    return res
v = calculate_local(D)
v

array([[ 1.52504049e-05,  1.52504049e-05,  5.44849645e-42,
         5.27080999e-44],
       [ 1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
        -1.00000000e+00]])

# Methods for finding global normed weights of alternatives

In [6]:
# distributive synthesis approach
def distributed_synthesis(w_crit, v):
    """
    Computes global weights
    """
    n, m = v.shape
    w_glob = np.empty(n)
    for i in range(n):
        # compute 'w_globe' components one by one
        r = v[i, :] / np.sum(v[:, i])
        r = r.reshape(m, 1)
        print("w_crit.shape =", w_crit.shape)
        print("r.shape =", r.shape)
        w_glob[i] = np.dot(w_crit.T, r)
    return w_glob

In [7]:
print(distributed_synthesis(w_crit, v).shape)
print(distributed_synthesis(w_crit, v))

NameError: name 'w_crit' is not defined

In [ ]:
w_glob = distributed_synthesis(w_crit, v)
print(np.sum(w_glob))